# External Data Retrieval
Uses external APIs for more data. Saves data for further analysis and offline use.

In [1]:
# All imports
import pandas as pd
import numpy as np
import torch
import pickle
from pathlib import Path
import utils

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

## Data retrieved from the Ensembl API

In [2]:
# Load DataFrames
train_df = pd.read_csv(
    utils.RAW_TRAIN_PATH,
    usecols=["accession", "scoreset", "ensp", "pos", "ref_long", "ref_short", "alt_long", "alt_short"],
    dtype={"accession": str, "scoreset": str, "ensp": str, "pos": int, "ref_long": str, "ref_short": str, "alt_long": str, "alt_short": str}
)
test_df = pd.read_csv(
    utils.RAW_TEST_PATH,
    usecols=["accession", "ensp", "pos", "ref_long", "ref_short", "alt_long", "alt_short"],
    dtype={"accession": str, "ensp": str, "pos": int, "ref_long": str, "ref_short": str, "alt_long": str, "alt_short": str}
)

In [ ]:
# Ensembl Protein ID to Sequence mapping
train_ensp_sequence_map: dict[str, str] = {}
test_ensp_sequence_map: dict[str, str] = {}
train_sequence_series: pd.Series = train_df["ensp"].apply(lambda ensp: utils.get_full_sequence(ensp, train_ensp_sequence_map))
test_sequence_series: pd.Series = test_df["ensp"].apply(lambda ensp: utils.get_full_sequence(ensp, test_ensp_sequence_map))

In [ ]:
# Save the mappings using pickle
utils.dict_to_pickle(train_ensp_sequence_map, utils.ENSP_SEQUENCE_MAP_PATH)
utils.dict_to_pickle(test_ensp_sequence_map, utils.ENSP_SEQUENCE_MAP_PATH)

In [ ]:
# Ensembl Variant Effect Predictor (VEP)
train_vep_data: list[dict] = utils.vep_from_pickle(utils.TRAIN_VEP_DATA_PATH)

In [ ]:
# Example usage of VEP data
# NOTE: "strand" is always 1. To get actual strand, use ["transcript_consequences"][0]["strand"]
print(len(train_vep_data))
print(train_vep_data[0].keys())

print(train_vep_data[0]["most_severe_consequence"])
print(train_vep_data[0]["transcript_consequences"][0]["impact"])
print(train_vep_data[0]["transcript_consequences"][0]["biotype"])
print(train_vep_data[0]["transcript_consequences"][0]["consequence_terms"])
print(train_vep_data[0]["transcript_consequences"][0]["strand"])

variant_types: set[str] = set()
impact_types: set[str] = set()
bio_types: set[str] = set()
consequence_types: list[str] = []
for row in train_vep_data:
    variant_types.add(row.get("most_severe_consequence", "error"))
    impact_types.add(row["transcript_consequences"][0].get("impact", "error"))
    bio_types.add(row["transcript_consequences"][0].get("biotype", "error"))
    for c in row["transcript_consequences"]:
        for consequence in c.get("consequence_terms", ["error"]):
            if consequence not in consequence_types:
                consequence_types.append(consequence)
print(variant_types)
print(impact_types)
print(bio_types)
print(consequence_types)

## Data retrieved from the MaveDB API

# Embeddings from ESM C

In [3]:
ensp_sequence_map = utils.pickle_to_dict(utils.ENSP_SEQUENCE_MAP_PATH)

In [4]:
ensp_embeddings_map: dict[str, torch.Tensor] = {}
num_embeddings = 0
for ensp, seq in ensp_sequence_map.items():
    ensp_embeddings_map[ensp] = utils.get_embedding(seq)
    num_embeddings += 1
    print(f"Processed {num_embeddings} embeddings")

torch.Size([36, 1, 807, 1152])
Processed 1 embeddings
torch.Size([36, 1, 165, 1152])
Processed 2 embeddings
torch.Size([36, 1, 1865, 1152])
Processed 3 embeddings
torch.Size([36, 1, 678, 1152])
Processed 4 embeddings
torch.Size([36, 1, 697, 1152])
Processed 5 embeddings
torch.Size([36, 1, 362, 1152])
Processed 6 embeddings
torch.Size([36, 1, 200, 1152])
Processed 7 embeddings
torch.Size([36, 1, 190, 1152])
Processed 8 embeddings
torch.Size([36, 1, 320, 1152])
Processed 9 embeddings


KeyboardInterrupt: 

In [ ]:
print(len(ensp_embeddings_map))
for ensp in ensp_embeddings_map:
    print(ensp, ensp_embeddings_map[ensp].shape)

In [ ]:
utils.dict_to_pickle(ensp_embeddings_map, utils.ENSP_EMBEDDINGS_MAP_PATH)